In [11]:
from pytket.extensions.nexus import Nexus
from datetime import datetime
from pytket.extensions.nexus import QuantinuumConfig
from pytket.extensions.nexus import NexusBackend
import pytket as tk
from pytket import passes as tkp
from pytket import circuit as tkc

from pytket.circuit.display import render_circuit_jupyter as print_circ

import numpy as np
from pytket import Circuit
from pytket import OpType
from pytket.circuit.display import render_circuit_jupyter

In [2]:
def cliff_t_rebase() -> tkp.BasePass:
    """Pass to convert single-qubit gates to the Clifford+T gateset.

    Concretely, single-qubit gates will be one of H, Z, S or T. Could also be
    updraded to handle X and V. Have a try!

    pyTKET won't do this for you automatically, because this is not a universal
    gateset (only approximately universal).

    For our purpose, if the decomposition is not exact, we raise an error
    """
    cx_replacement = tk.Circuit(2).CX(0, 1)
    def tk1_replacement(a, b, c, eps=1e-6):
        # make sure the phases are in the range [0, 4)
        a, b, c = a % 4, b % 4, c % 4
        ret = tk.Circuit(1)
        def add_phase(f: float):
            while f > eps:
                if f + eps > 1.:
                    ret.Z(0)
                    f -= 1
                elif f + eps > 0.5:
                    ret.S(0)
                    f -= 0.5
                elif f + eps > 0.25:
                    ret.T(0)
                    f -= 0.25
                else:
                    break
            return f
        rest_c = add_phase(c)
        ret.H(0)
        rest_b = add_phase(b)
        ret.H(0)
        rest_a = add_phase(a)

        if abs(rest_a) > eps or abs(rest_b) > eps or abs(rest_c) > eps:
            raise ValueError("Phases are not multiples of pi/4")
        return ret

    return tkp.RebaseCustom(
        {tk.OpType.CX, tk.OpType.H, tk.OpType.S, tk.OpType.T},
        cx_replacement=cx_replacement,
        tk1_replacement=tk1_replacement
    )

In [89]:
def cliff_t_rebase_2() -> tkp.BasePass:
    """Pass to convert single-qubit gates to the Clifford+T gateset.

    Concretely, single-qubit gates will be one of H, Z, S or T. Could also be
    updraded to handle X and V. Have a try!

    pyTKET won't do this for you automatically, because this is not a universal
    gateset (only approximately universal).

    For our purpose, if the decomposition is not exact, we raise an error
    """
    cx_replacement = tk.Circuit(2).CX(0, 1)
    def tk1_replacement(a, b, c, eps=1e-6):
        # make sure the phases are in the range [0, 4)
        a, b, c = a % 4, b % 4, c % 4
        ret = tk.Circuit(1)
        def add_phase(f: float):
            while f > eps:
                if f + eps > 1.:
                    ret.Z(0)
                    f -= 1
                elif f + eps > 0.5:
                    ret.S(0)
                    f -= 0.5
                elif f + eps > 0.25:
                    ret.T(0)
                    f -= 0.25
                else:
                    break
            return f
        rest_c = add_phase(c)
        ret.H(0)
        rest_b = add_phase(b)
        ret.H(0)
        rest_a = add_phase(a)

        if abs(rest_a) > eps or abs(rest_b) > eps or abs(rest_c) > eps:
            raise ValueError("Phases are not multiples of pi/4")
        return ret

    return tkp.RebaseCustom(
        {tk.OpType.X, tk.OpType.Y, tk.OpType.Z},
        cx_replacement=cx_replacement,
        tk1_replacement=tk1_replacement
    )

In [4]:
def get_res(name, my_circuit, shots):
    configuration = QuantinuumConfig(device_name="H1-1LE", user_group="iQuHACK_2024")
    nexus = Nexus()
    my_project = nexus.new_project(name=name)
    # Then we'll create a NexusBackend using our config and the project we created
    backend = NexusBackend(configuration, project=my_project)
    compiled_circuit = backend.get_compiled_circuit(my_circuit, optimisation_level=2)
    handle = backend.process_circuit(compiled_circuit, n_shots=shots)
    backend.circuit_status(handle)
    result = backend.get_result(handle)
    result.get_distribution()


In [5]:
num_qbit = 3
c = Circuit(num_qbit).CX(0,1).H(1).T(1).CX(1,2).H(1).H(2).T(0).T(2)

print_circ(c)
def H_gadgets(c:Circuit):
    d = Circuit(c.n_qubits)
    num_H = c.n_gates_of_type(OpType.H)
    ancilla_q = d.add_q_register("a", num_H)
    cr = d.add_c_register("c", num_H)
    count_ancilla = 0
    for g in c:
        if g.op.type == OpType.H:
            print("find H gate")
            count_ancilla += 1
            anc_act = ancilla_q[count_ancilla-1]
            d.H(anc_act)
            d.S(g.qubits[0])
            d.S(anc_act)
            d.CX(g.qubits[0], anc_act)
            d.Sdg(anc_act)
            d.CX(anc_act,g.qubits[0])
            d.CX(g.qubits[0], anc_act)
            d.Measure(anc_act,cr[count_ancilla-1])
            d.X(*g.qubits,condition_bits=[cr[count_ancilla-1]], condition_value=1)
        else:
            d.add_gate(g.op, g.qubits)
    return d
print_circ(H_gadgets(c))

find H gate
find H gate
find H gate


In [42]:
# num_qbit = 3
# c = Circuit(num_qbit).CX(0,1).H(1).T(1).CX(1,2).H(1).H(2).T(0).T(2)
c = Circuit(2)
# We build a 3qb gate with the following diagonal
# Note that the resulting diagonal matrix must be unitary, so all values must have amplitude 1.
diag = np.exp(1j * np.pi * np.arange(4) / 4)
print(diag)
diag_box = tkc.DiagonalBox(diag)
# c.add_gate(diag_box, [0, 1, 2])
c.H(1)
c.add_gate(diag_box, [0, 1])
# This gate is a bit too complex too render nicely graphically, so the renderer just shows
# us a very helpful grey box... fair enough
print_circ(c)

[ 1.00000000e+00+0.j          7.07106781e-01+0.70710678j
  6.12323400e-17+1.j         -7.07106781e-01+0.70710678j]


In [43]:
CH = H_gadgets(c)
print_circ(CH)

find H gate


In [45]:
d = Circuit(CH.n_qubits)
    # cr = d.add_c_register("c", num_H)
num_measurement = CH.n_gates_of_type(OpType.Measure)
ancilla_q = d.add_q_register("a", num_measurement)
for g in CH:
        if (g.op.type == OpType.Measure):
            # print("find X measurement gate")
            print(g.op.type, g.qubits)
        elif (g.op.type == OpType.Conditional):
            print(g.op.type, g.qubits)
        elif g.op.type == OpType.DiagonalBox:
            d.add_gate(g.op, g.qubits)
        elif (g.op.type != OpType.Measure) and (g.op.type != OpType.Conditional):
            d.add_gate(g.op, g.qubits)

OpType.Measure [a[0]]
OpType.Conditional [q[1]]


In [63]:
X = [[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]]
diag_mat = np.diag(diag)
diag_dagger = np.conj(diag_mat).T
matrix_prepare = np.matmul(diag_mat, np.matmul(X, diag_dagger))

In [61]:
diag_mat

array([[ 1.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
         0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
       [ 0.00000000e+00+0.j        ,  7.07106781e-01+0.70710678j,
         0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
       [ 0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
         6.12323400e-17+1.j        ,  0.00000000e+00+0.j        ],
       [ 0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
         0.00000000e+00+0.j        , -7.07106781e-01+0.70710678j]])

In [64]:
matrix_prepare

array([[1.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 1.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.70710678-0.70710678j],
       [0.        +0.j        , 0.        +0.j        ,
        0.70710678+0.70710678j, 0.        +0.j        ]])

In [65]:
from pytket.circuit import Unitary2qBox

In [ ]:
d.X(*g.qubits,condition_bits=[cr[count_ancilla-1]], condition_value=1)

In [66]:
new_matrix = Unitary2qBox(matrix_prepare)

Unitary2qBox

In [68]:
print_circ(d)

In [90]:
dummy = Circuit(2)
dummy.add_gate(new_matrix, [0,1])
print_circ(dummy)

In [91]:
box_decomp = tkp.SequencePass([tkp.DecomposeBoxes(), cliff_t_rebase_2()])
# cliff_opt = tkp.SequencePass([tkp.CliffordSimp(), cliff_t_rebase()])
# cliff_opt.apply(c)
box_decomp.apply(dummy)
# cliff_opt = tkp.SequencePass([tkp.CliffordSimp(), cliff_t_rebase_2()])
# # cliff_opt.apply(c)
# box_decomp.apply(dummy)
print_circ(dummy)

In [88]:
print_circ(d)

In [44]:
for g in CH:
    print(g.op.type, g.qubits)

OpType.H [a[0]]
OpType.S [q[1]]
OpType.S [a[0]]
OpType.CX [q[1], a[0]]
OpType.Sdg [a[0]]
OpType.CX [a[0], q[1]]
OpType.CX [q[1], a[0]]
OpType.Measure [a[0]]
OpType.Conditional [q[1]]
OpType.DiagonalBox [q[0], q[1]]


In [36]:
def prepare_X_pushing(c:Circuit):
    d = Circuit(c.n_qubits)
    # cr = d.add_c_register("c", num_H)
    num_measurement = c.n_gates_of_type(OpType.Measure)
    ancilla_q = d.add_q_register("a", num_measurement)
    for g in c:
        if (g.op.type == OpType.Measure):
            print(g.op.type, g.qubits)
        elif (g.op.type == OpType.Conditional):
            print(g.op.type, g.qubits)
        elif g.op.type == OpType.DiagonalBox:
            d.add_gate(g.op, g.qubits)
        elif (g.op.type != OpType.Measure) and (g.op.type != OpType.Conditional):
            d.add_gate(g.op, g.qubits)
    return d
print_circ(X_pushing(CH))


find X measurement gate


In [81]:
c = tk.Circuit(3)

# We build a 3qb gate with the following diagonal
# Note that the resulting diagonal matrix must be unitary, so all values must have amplitude 1.
diag = np.exp(1j * np.pi * np.arange(8) / 4)
diag_dagger = np.exp(-1j * np.pi * np.arange(8) / 4)
# print(diag)
diag_box = tkc.DiagonalBox(diag)
c.add_gate(diag_box, [0, 1, 2])
box_decomp = tkp.SequencePass([tkp.DecomposeBoxes(), cliff_t_rebase_2()])
# cliff_opt = tkp.SequencePass([tkp.CliffordSimp(), cliff_t_rebase()])

cliff_opt = tkp.SequencePass([tkp.CliffordSimp(), cliff_t_rebase_2()])
# cliff_opt.apply(c)
box_decomp.apply(c)
# cliff_opt.apply(c)

[ 1.00000000e+00+0.00000000e+00j  7.07106781e-01+7.07106781e-01j
  6.12323400e-17+1.00000000e+00j -7.07106781e-01+7.07106781e-01j
 -1.00000000e+00+1.22464680e-16j -7.07106781e-01-7.07106781e-01j
 -1.83697020e-16-1.00000000e+00j  7.07106781e-01-7.07106781e-01j]


True

In [39]:
d = Circuit(c.n_qubits)
    # cr = d.add_c_register("c", num_H)
num_measurement = CH.n_gates_of_type(OpType.Measure)
ancilla_q = d.add_q_register("a", num_measurement)
for g in CH:
        if (g.op.type == OpType.Measure):
            # print("find X measurement gate")
            print(g.op.type, g.qubits)
        elif (g.op.type == OpType.Conditional):
            print(g.op.type, g.qubits)
        elif g.op.type == OpType.DiagonalBox:
            d.add_gate(g.op, g.qubits)
        elif (g.op.type != OpType.Measure) and (g.op.type != OpType.Conditional):
            d.add_gate(g.op, g.qubits)

OpType.Measure [a[0]]
OpType.Conditional [q[1]]


In [40]:
print_circ(d)

In [83]:
# H_c = H_gadgets(c)
# cliff_opt.apply(c)
print_circ(c)
print_circ(c.dagger())

In [43]:
for g in H_c:
    print(g.op.type, g.qubits)

OpType.H [a[0]]
OpType.DiagonalBox [q[0], q[1], q[2]]
OpType.S [a[0]]
OpType.S [q[1]]
OpType.CX [q[1], a[0]]
OpType.Sdg [a[0]]
OpType.CX [a[0], q[1]]
OpType.CX [q[1], a[0]]
OpType.Measure [a[0]]
OpType.Conditional [q[1]]
OpType.DiagonalBox [q[0], q[1], q[2]]


In [51]:
box_decomp = tkp.SequencePass([tkp.DecomposeBoxes(), cliff_t_rebase()])
# cliff_opt = tkp.SequencePass([tkp.CliffordSimp(), cliff_t_rebase()])
# cliff_opt.apply(c)
box_decomp.apply(H_c)
print_circ(H_c)

In [52]:
cliff_opt = tkp.SequencePass([tkp.CliffordSimp(), cliff_t_rebase()])
cliff_opt.apply(H_c)

True

In [53]:
print_circ(H_c)

In [17]:
print_circ(c_dg)